# Code

In [ ]:
import openseespy.opensees as ops
import opsvis as opsv
import os
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

if not os.path.exists('Data'): 
    os.mkdir('Data')

ops.wipe()  # Clear opensees model

#----------------------------------------------------------- Units and constants-----------------------------------------------------------------------------------------#

# Defining base units for the simulation.
m = 1.0  # Meter (base unit for length)
s = 1.0  # Second (base unit for time)
kg = 1.0  # Kilogram (base unit for mass)

# Derived units for force and pressure.
N = kg * m / s ** 2  # Newton (unit of force)
Pa = N / m ** 2  # Pascal (unit of pressure)

# Additional conversions for different units commonly used in structural engineering.
inches = 0.0254 * m  # Conversion factor for inches to meters.
ft = 12 * inches  # Conversion factor for feet to meters.
kip = 4448.2216152548 * N  # Conversion factor for kips (1000 pounds-force) to Newtons.
ksi = 6.895 * 10 ** 6 * Pa  # Conversion factor for ksi (1000 psi) to Pascals.

g = 9.81 * N / kg

#------------------------------------------------------------Define Widgets/Inputs------------------------------------------------------------------------#

beam_length = widgets.FloatText(
    value= 42.0 * ft,
    description='Beam length (m):',
    disabled=False,
    style={'description_width': 'initial'}
)

column_length = widgets.FloatText(
    value= 36.0 * ft,
    description='Column height (m):',
    disabled=False,
    style={'description_width': 'initial'}
)

bBeam = widgets.FloatText(
    value=1,
    description='Beam breadth (m):',
    disabled=False,
    style={'description_width': 'initial'}
)

dBeam = widgets.FloatText(
    value=1,
    description='Beam depth (m):',
    disabled=False,
    style={'description_width': 'initial'}
)

bCol = widgets.FloatText(
    value=1,
    description='Column breadth (m):',
    disabled=False,
    style={'description_width': 'initial'}
)

dCol = widgets.FloatText(
    value=1,
    description='Column depth (m):',
    disabled=False,
    style={'description_width': 'initial'}
)


E = widgets.FloatText(
    value=200 * 10 ** 9,
    description='Elastic modulus (Pa):',
    disabled=False,
    style={'description_width': 'initial'}
)

Py = widgets.FloatText(
    value=-4000.0 * kip,
    description='Vertical load (N):',
    disabled=False,
    style={'description_width': 'initial'}
)

Node1_restraint = widgets.Dropdown(
    options=[('Fixed', [1, 1, 1]), ('Pinned', [1, 1, 0])],
    description = 'Node 1 restraint',
    disabled = False,
    style={'description_width': 'initial'}
)

Node2_restraint = widgets.Dropdown(
    options=[('Fixed', [1, 1, 1]), ('Pinned', [1, 1, 0])],
    description = 'Node 2 restraint',
    disabled = False,
    style={'description_width': 'initial'}
)

Node3_restraint = widgets.Dropdown(
    options=[('Free', [0, 0, 0]), ('Rotationally restrained', [0, 0, 1])],
    description = 'Node 3 restraint',
    disabled = False,
    style={'description_width': 'initial'}
)

Node4_restraint = widgets.Dropdown(
    options=[('Free', [0, 0, 0]), ('Rotationally restrained', [0, 0, 1])],
    description = 'Node 4 restraint',
    disabled = False,
    style={'description_width': 'initial'}
)


EQ_file = widgets.Text(
    value = 'BM68elc.acc',
    description = 'File path'
)

out_model = widgets.Output(
    layout={'border': '1px solid black'}
)

out_cross_sections = widgets.Output(
    layout={'border': '1px solid black'}
)

disp = widgets.FloatText(
    value=1,
    description='Horizontal displacement (m):',
    disabled=False,
    style={'description_width': 'initial'}
)

analysis = widgets.Dropdown(
    options=['Push Over', 'Earthquake'],
    value='Push Over',
    description='Analysis:',
    disabled=False,
)

output_analysis = widgets.Output(
    layout={'border': '1px solid black'}
    )

# https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20Layout.html#the-layout-attribute
layout = widgets.Layout(
    border='2px solid red',
    # justify_content = 'center'
    )


button = widgets.Button(description="Run Analysis", disabled = False, layout = layout)
#--------------------------------------------------------------------------------------------------------------------------------------------------------#



def model_definition(change):

    button.disabled = False

    ops.wipe() # Clear opensees model
    ops.model('basic','-ndm',2,'-ndf',3)     #  # 2D model with 3 degrees of freedom per node.

    # define GEOMETRY -------------------------------------------------------------

    # Nodal coordinates
    n1 = (0.0, 0.0) # Use floating point values
    n2 = (beam_length.value, 0.0)
    n3 = (0.0, column_length.value)
    n4 = (beam_length.value, column_length.value)


    # Cross-sectional dimensions
    ABeam = bBeam.value * dBeam.value # Area
    ACol = bCol.value * dCol.value # Area

    # Create nodes
    nodal_crds = (n1, n2, n3, n4)
    for nodeTag, crds in enumerate(nodal_crds, start = 1): # https://docs.python.org/3/library/functions.html#enumerate
        ops.node(nodeTag, *crds)
    
    # Single point constraints -- Boundary Conditions
    ops.fix(1, *Node1_restraint.value)     #  node DX DY RZ
    ops.fix(2, *Node2_restraint.value)     #  node DX DY RZ
    ops.fix(3, *Node3_restraint.value)
    ops.fix(4, *Node4_restraint.value)

    # nodal masses:
    mass_x = abs(Py.value) / (2 * g) # kg
    ops.mass(3,mass_x,0.,0.)     #  node , Mx My Mz, Mass=Weight/g.
    ops.mass(4,mass_x,0.,0.)

    # Define ELEMENTS -------------------------------------------------------------
    # define geometric transformation: performs a linear geometric transformation of beam stiffness and resisting force from the basic system to the global-coordinate system
    ops.geomTransf('Linear', 1)     #  associate a tag to transformation

    # Moment of Inertia
    IzBeam = (bBeam.value * dBeam.value ** 3) / 12 # Beam
    IzCol = (bCol.value * dCol.value ** 3) / 12 # Column

    # connectivity: (make A very large, 10e6 times its actual value)
    ops.element('elasticBeamColumn', 1, 1, 3, ACol * 10 ** 6, E.value, IzCol, 1)     #  element elasticBeamColumn eleTag iNode jNode A E Iz transfTag
    ops.element('elasticBeamColumn', 2, 2, 4, ACol * 10 ** 6, E.value, IzCol, 1)
    ops.element('elasticBeamColumn', 3, 3, 4, ABeam * 10 ** 6, E.value, IzBeam, 1)

    plot_model()
    plot_cross_sections(change)
    


@out_model.capture(clear_output = True, wait = True)
def plot_model():
    opsv.plot_model()
    plt.title('Model')
    plt.ylabel('Height (m)')
    plt.show()

@out_cross_sections.capture(clear_output = True, wait = True)
def plot_cross_sections(change):

    fig, axes = plt.subplots(1, 2)
    axes[0].set_aspect(aspect = 'equal', adjustable = 'box')
    axes[0].plot([0, 0, bBeam.value, bBeam.value, 0], [0, dBeam.value, dBeam.value, 0, 0])
    axes[0].set_title('Beam Cross-section')
    axes[0].set_xlabel('Breadth (m)')
    axes[0].set_ylabel('Depth (m)')
    # xTickData = ax.get_xticks()
    axes[0].margins(1.0) # https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.margins.html#matplotlib.axes.Axes.margins
    axes[0].grid()

    axes[1].set_aspect(aspect = 'equal', adjustable = 'box')
    axes[1].plot([0, 0, bCol.value, bCol.value, 0], [0, dCol.value, dCol.value, 0, 0])
    axes[1].set_title('Column Cross-section')
    axes[1].set_xlabel('Breadth (m)')
    axes[1].margins(1.0) # https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.margins.html#matplotlib.axes.Axes.margins
    axes[1].grid()
    plt.show()


def pushOverAnalysis(disp):

    # Load definition (Lateral loads)

    # define LATERAL load -------------------------------------------------------------
    # Lateral load pattern
    ops.timeSeries('Linear', 2)     # timeSeries Linear 2;
    # define Load Pattern
    ops.pattern('Plain', 2, 2) # 
    ops.load(3, 1.0, 0.0, 0.0)     #  node , FX FY MZ -- representative lateral load at top nodes
    ops.load(4, 1.0, 0.0, 0.0)     #  place 1/2 of the weight for each node to get shear coefficient


    # Load analysis (Lateral loads)
    # Note we do not wipe the analysis
    print(f'Horizontal displacement at top of column is set to be: {disp} metres')
    steps = 100 # Must be an integer
    incr = disp / steps
    ops.integrator('DisplacementControl', 3, 1, incr)     #  switch to displacement control, for node 3, dof 1, increment
    ops.analyze(steps)

    # Output
    sfac = opsv.plot_defo(sfac = 1)
    plt.title(f'Deformed shape of portal frame (Scale factor: {sfac})')

    opsv.plot_loads_2d(nep=17, sfac=False, fig_wi_he=False, fig_lbrt=False, 
                   fmt_model_loads={'color': 'black', 'linestyle': 'solid', 'linewidth': 1.2, 'marker': '', 'markersize': 1}, 
                   node_supports=True, truss_node_offset=0, ax=False)
    
    plt.title('Loads applied to Portal Frame (N)')
    plt.xlabel('Metres')
    plt.ylabel('Metres')
    plt.show()
    
    info = ops.printModel()

def dynamicAnalysis(EQ_file):
    
    # Load Definition (Lateral loads)
    # create load pattern
    accelSeries  = 900
    ops.timeSeries( 'Path', accelSeries, '-dt', 0.01, '-filePath', EQ_file, '-factor', 1)     #  define acceleration vector from file (dt=0.01 is associated with the input file gm)
    ops.pattern('UniformExcitation', 2, 1, '-accel', accelSeries)     #  define where and how (pattern tag, dof) acceleration is applied
    ops.rayleigh(0.0, 0.0, 0.0, 2 * 0.02 / np.pow(ops.eigen('-fullGenLapack',1)[0], 0.5))

    # Analysis Generation (Lateral loads)
    ops.wipeAnalysis()     #  clear previously-define analysis parameters
    ops.constraints('Plain')     #  how it handles boundary conditions
    ops.numberer('Plain')     #  renumber dofs to minimize band-width (optimization), if you want to
    ops.system('BandGeneral')     #  how to store and solve the system of equations in the analysis
    ops.test('NormDispIncr',1.0e-8,10)     #  determine if convergence has been achieved at the end of an iteration step
    ops.algorithm('Newton')     #  use Newtons solution algorithm: updates tangent stiffness at every iteration
    ops.integrator('Newmark',0.5,0.25)     #  determine the next time step for an analysis
    ops.analysis('Transient')     #  define type of analysis: time-dependent
    ops.analyze(1000, 0.02)     #  apply 1000 0.02-sec time steps in analysis

    # Output
    ops.wipe()
    plt.close('all')

    # Load the data
    fname3 = 'Data/DispFreeEx2_EQ.out'
    dataDFree = np.loadtxt(fname3)

    # Create a figure and axes instances
    fig, ax = plt.subplots(1, 1)

    # Plot on the first axis
    ax.set_title('Ex2a. Elastic Portal Frame EQ Analysis')
    ax.grid(True)
    ax.plot(dataDFree[:, 0], dataDFree[:, 1])
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Free-Node Disp. (m)')

    plt.show()

    # End of script
    print('End of Run')


def run_analysis(Py, beam_length, disp, analysis):

    if analysis == 'Push Over':
        name_suffix = 'Push.out'
    else:
        name_suffix = 'EQ.out'

    # Define RECORDERS -------------------------------------------------------------
    # https://openseespydoc.readthedocs.io/en/latest/src/nodeRecorder.html#node-recorder-command
    # recorder('Node', '-file', filename, '-time', '-node', *nodeTags=[], '-dof', *dofs=[], respType)
    ops.recorder('Node', '-file', 'Data/DispFreeEx2_' + name_suffix, '-time', '-node', 3, 4, '-dof', 1, 2, 3, 'disp')     #  displacements of free nodes
    ops.recorder('Node', '-file','Data/DispBaseEx2_' + name_suffix,'-time','-node',1, 2, '-dof', 1, 2, 3, 'disp')     #  displacements of support nodes
    ops.recorder('Node', '-file', 'Data/ReacBaseEx2_' + name_suffix, '-time', '-node', 1, 2,'-dof', 1, 2, 3, 'reaction')     #  support reaction
    # https://openseespydoc.readthedocs.io/en/latest/src/elementRecorder.html#element-recorder-command
    # recorder('Element', '-file', filename, '-time', '-ele', *eleTags=[], '-eleRange', startEle, endEle, '-region', regionTag, *args)
    ops.recorder('Element', '-file', 'Data/FColEx2_' + name_suffix, '-time', '-ele', 1, 2, 'globalForce')     #  element forces -- column
    ops.recorder('Element', '-file', 'Data/FBeamEx2_' + name_suffix, '-time', '-ele', 3, 'globalForce')     #  element forces -- beam



    # Load Definition (gravity loads)
    ops.timeSeries('Linear',1)     # timeSeries Linear 1;
    # define Load Pattern
    ops.pattern('Plain',1,1)
    w = Py / beam_length
    ops.eleLoad('-ele',3,'-type','-beamUniform', w)     # w distributed superstructure-weight on beam

    # Analysis generation (gravity loads)
    ops.wipeAnalysis()     # adding this to clear Analysis module 
    ops.constraints('Plain')     #  how it handles boundary conditions
    ops.numberer('Plain')     #  renumber dofs to minimize band-width (optimization), if you want to
    ops.system('BandGeneral')     #  how to store and solve the system of equations in the analysis
    ops.test('NormDispIncr',1.0e-8,6)     #  determine if convergence has been achieved at the end of an iteration step
    ops.algorithm('Newton')     #  use Newtons solution algorithm: updates tangent stiffness at every iteration
    ops.integrator('LoadControl',0.1)     #  determine the next time step for an analysis,   apply gravity in 10 steps
    ops.analysis('Static')     #  define type of analysis static or transient
    ops.analyze(10)     #  perform gravity analysis
    ops.loadConst('-time',0.0)     #  hold gravity constant and restart time

    # print(f'The distributed superstructure-weight on the beam is {w:0.2f} N/m. This is equivalent to {w * 5.710144071 * 10 ** -6:0.2f} kips/inch')

    if analysis == 'Push Over':
        pushOverAnalysis(disp)
    else:
        dynamicAnalysis(EQ_file.value)


@output_analysis.capture(clear_output = True)
def on_button_clicked(b):
    run_analysis(Py.value, beam_length.value, disp.value, analysis.value)
    b.disabled = True



model_definition('dummy')
button.on_click(on_button_clicked)

VBox_EQ = widgets.VBox([EQ_file, column_length, beam_length, bBeam, dBeam, bCol, dCol, Node1_restraint, Node2_restraint, Node3_restraint, Node4_restraint, E, Py])
# VBox_EQ_1 = widgets.VBox([EQ_file, column_length, beam_length, bBeam, dBeam, bCol, dCol])
# VBox_EQ_2 = widgets.VBox([Node1_restraint, Node2_restraint, Node3_restraint, Node4_restraint, E, Py])
# HBox_EQ = widgets.HBox([VBox_EQ_1, VBox_EQ_2], layout = widgets.Layout(justify_content = 'center'))
VBox_pushOver = widgets.VBox([disp, column_length, beam_length, bBeam, dBeam, bCol, dCol, Node1_restraint, Node2_restraint, Node3_restraint, Node4_restraint, E, Py])
# VBox_pushOver_1 = widgets.VBox([disp, column_length, beam_length, bBeam, dBeam, bCol, dCol])
# VBox_pushOver_2 = widgets.VBox([Node1_restraint, Node2_restraint, Node3_restraint, Node4_restraint, E, Py])
# HBox_pushOver = widgets.HBox([VBox_pushOver_1, VBox_pushOver_2], layout = widgets.Layout(justify_content = 'center'))

HBox = widgets.HBox([out_model, out_cross_sections], layout = widgets.Layout(justify_content = 'center'))
# stack = widgets.Stack([HBox_pushOver, HBox_EQ], selected_index=0)
stack = widgets.Stack([VBox_pushOver, VBox_EQ], selected_index=0)
# dropdown = widgets.Dropdown(options=['button', 'slider'])
widgets.jslink((analysis, 'index'), (stack, 'selected_index'))
VBox_out = widgets.VBox([analysis, stack])


beam_length.observe(model_definition, names='value')
column_length.observe(model_definition, names='value')
bBeam.observe(model_definition, names='value')
dBeam.observe(model_definition, names='value')
bCol.observe(model_definition, names='value')
dCol.observe(model_definition, names='value')
Node1_restraint.observe(model_definition, names='value')
Node2_restraint.observe(model_definition, names='value')
Node3_restraint.observe(model_definition, names='value')
Node4_restraint.observe(model_definition, names='value')
disp.observe(model_definition, names='value')
analysis.observe(model_definition, names='value')
E.observe(model_definition, names='value')
Py.observe(model_definition, names='value')
EQ_file.observe(model_definition, names='value')
   
    





    








# Inputs

In [4]:
display(VBox_out, button, HBox, output_analysis)

Button(description='Run Analysis', layout=Layout(border_bottom='2px solid red', border_left='2px solid red', b…

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…